<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Scrape all Artists & Store in SQLite DB  </span>

In [21]:
%run crawlers/artist_scraping.ipynb
%run py_utils/scraping_utils.ipynb
%run py_utils/sqlite_utils.ipynb

def acquire_artists():
    return get_artists()
    
def store_artists(artist_data, con, cur):
    query = """CREATE TABLE IF NOT EXISTS artists(
                artist_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                artist_name TEXT NOT NULL, 
                artist_path TEXT NOT NULL,
                created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
                updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
                UNIQUE(artist_name, artist_path)
                )"""    
    run_query(cur, con, query)

    for artist in artist_data:
        try:
            query = "INSERT INTO artists (artist_name, artist_path) values ( ?, ? )"
            run_query(cur, con, query, values=(artist, artist_data[artist]))       

        except Exception as exc:
            print("! Exception: {}".format(exc))

        else:
            print(">> {} added to database.".format(artist))    

con, cur = connect_sqlite("../database/azlyrics.db")  
artist_data = acquire_artists()  
store_artists(artist_data, con, cur)
con.close()


>> Scraping URL:  https://www.azlyrics.com/a.html
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.artist_path
! Exception: UNIQUE constraint failed: artists.artist_name, artists.ar

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Artist Selection Interface </span>

In [14]:
import string
%run py_utils/widgets_utils.ipynb
%run py_utils/sqlite_utils.ipynb

user_selection = []
alphabets = [letter for letter in string.ascii_uppercase]
alphabets.append('#')

con, cur = connect_sqlite("../database/azlyrics.db")
cur.execute("SELECT artist_name FROM artists")
all_artists = [data[0] for data in cur.fetchall()]

def retrieve_alphabet(change):
    filter_artist = []
    selected_item = change.new    
    widget_output(output, "Alphabet {} selected".format(selected_item))  
    
    for artist in all_artists:
        if selected_item == artist[0].upper(): filter_artist.append(artist)            
        else:
            if artist[0].upper() not in alphabets and selected_item == '#': filter_artist.append(artist)    
    alphabet_dropdown.value = selected_item
    artist_dropdown.options = filter_artist
    artist_dropdown.value = filter_artist[0]
    
def retrieve_artist(change):
    widget_output(output, f"{change.new} selected")
    
def retrieve_data(_):    
    if artist_dropdown.value != '': user_selection.append(artist_dropdown.value)
    widget_output(output, 'Selection: • '+' • '.join(user_selection))

def clear_data(_):
    try:
        user_selection.pop()
        widget_output(output, 'Selection: '+', '.join(user_selection))
        
    except Exception as exc: 
        widget_output(output, '! Exception: '+str(exc))
    
output = create_output()
widget_output(output, "Please select an Artist:")
alphabet_dropdown = assign_dropdown(alphabets, "Alphabets", arg_function=retrieve_alphabet, observe=1)
artist_dropdown = assign_dropdown([''], "Artists", arg_function=retrieve_artist, observe=1)
dropdowns = create_gui(alphabet_dropdown, artist_dropdown, num=[0,1], wd='300', widget_type='H', gui_display=0)

select_button = create_button("Add Artist to Selection", arg_function=retrieve_data)
clear_button = create_button("Clear Last Selection", arg_function=clear_data)
buttons = create_gui(select_button, clear_button, num=[0,1], wd='300', widget_type='H', gui_display=0)
create_gui(output, dropdowns, buttons, widget_type='V', gui_display=1) 


In [15]:
user_selection

['Kasabian']

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Create Albums and Tracks Table </span>

In [1]:
%run py_utils/sqlite_utils.ipynb

def create_album_table(con, cur):  
    query = """CREATE TABLE IF NOT EXISTS albums(
               album_id INTEGER PRIMARY KEY AUTOINCREMENT, 
               album_name TEXT NOT NULL, 
               album_year INTEGER,
               artist_id INTEGER, 
               created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
               updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
               FOREIGN KEY (artist_id) REFERENCES artists (artist_id),               
               UNIQUE(album_name, artist_id)
               )"""
    run_query(cur, con, query)
        
def create_track_table(con, cur):    
    query = """CREATE TABLE IF NOT EXISTS tracks(
                track_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                track_name TEXT NOT NULL, 
                track_lyrics TEXT, 
                track_credits TEXT, 
                track_path TEXT NOT NULL,
                artist_id INTEGER, 
                album_id INTEGER, 
                created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
                updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
                FOREIGN KEY (artist_id) REFERENCES artists (artist_id), 
                FOREIGN KEY (album_id) REFERENCES albums (album_id)             
                UNIQUE(track_name, track_path, artist_id, album_id)
                )"""
    run_query(cur, con, query)
    
    describe = cur.execute("PRAGMA table_info(tracks)")
    for columns in describe: print(columns)
    
con, cur = connect_sqlite("../database/azlyrics.db" ) 
create_album_table(con, cur)
create_track_table(con, cur)
con.close()


(0, 'track_id', 'INTEGER', 0, None, 1)
(1, 'track_name', 'TEXT', 1, None, 0)
(2, 'track_lyrics', 'TEXT', 0, None, 0)
(3, 'track_credits', 'TEXT', 0, None, 0)
(4, 'track_path', 'TEXT', 0, None, 0)
(5, 'artist_id', 'INTEGER', 0, None, 0)
(6, 'album_id', 'INTEGER', 0, None, 0)
(7, 'created_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)
(8, 'updated_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)


In [23]:
%run crawlers/album_scraping.ipynb
#TEST  "https://www.azlyrics.com/k/kasabian.html",  "https://www.azlyrics.com/q/qorygore.html"

start_urls = ["https://www.azlyrics.com/k/kasabian.html"]  ## 
artist_info = get_albums(start_urls) 


>> Scraping https://www.azlyrics.com/k/kasabian.html
! ValueError: invalid literal for int() with base 10: 'songs:'


In [28]:
print(len(artist_info['Kasabian']['track_names']), len(artist_info['Kasabian']['album_years']))

for i, track in enumerate(artist_info['Kasabian']['track_names']):
#     print(artist, '-', artist_info[artist]["album_names"][i], '-', artist_info[artist]["album_years"][i], '-')
    print(track, '-', artist_info['Kasabian']['album_names'][i], '-', artist_info['Kasabian']['album_years'][i], '-',)
    

96 96
Club Foot - album: Kasabian - 2005 -
Processed Beats - album: Kasabian - 2005 -
Reason Is Treason - album: Kasabian - 2005 -
I.D. - album: Kasabian - 2005 -
Orange (Interlude)(Instrumental) - album: Kasabian - 2005 -
L.S.F. (Lost Souls Forever) - album: Kasabian - 2005 -
Running Battle - album: Kasabian - 2005 -
Test Transmission - album: Kasabian - 2005 -
Pinch Roller (Interlude)(Instrumental) - album: Kasabian - 2005 -
Cutt Off - album: Kasabian - 2005 -
Butcher Blues - album: Kasabian - 2005 -
Ovary Stripe(Instrumental) - album: Kasabian - 2005 -
U Boat - album: Kasabian - 2005 -
Reason Is Treason (Jacknife Lee Version)(Hidden Track) - album: Kasabian - 2005 -
Empire - album: Empire - 2006 -
Shoot The Runner - album: Empire - 2006 -
Last Trip (In Flight) - album: Empire - 2006 -
Me Plus One - album: Empire - 2006 -
Sun Rise Light Flies - album: Empire - 2006 -
Apnoea - album: Empire - 2006 -
By My Side - album: Empire - 2006 -
Stuntman - album: Empire - 2006 -
Seek & Destroy -

In [6]:
%run crawlers/album_scraping.ipynb
# https://www.azlyrics.com/q/qorygore.html
root_url = "https://www.azlyrics.com"

def store_data(con, cur):
    for artist in artist_info:
        cur.execute("SELECT artist_id FROM artists WHERE artist_name=?", (artist,))
        artist_id = cur.fetchall()[0][0]
        
        for i, album in enumerate(artist_info[artist]['album_names']):
            pass
            try:
                query = "INSERT INTO albums (album_name, album_year, artist_id) values ( ?, ?, ? )"
                run_query(cur, con, query, (album, artist_info[artist]['album_years'][i], artist_id))
                print(">> Entries added:", album, " - ", artist_info[artist]['album_years'][i], " - ", artist_id)                  
            except Exception as exc: print("! Exception {}".format(exc))
        
        for i, track in enumerate(artist_info[artist]['track_names']):
            print(track)
            cur.execute("SELECT album_id FROM albums WHERE artist_id=?", (artist_id,))
            album_id = cur.fetchall()[0][0]
            track_url = (lambda url: '' if url == root_url else url) (artist_info[artist]["track_urls"][i])
            print(album_id)
            
#             try:
#                 query = "INSERT INTO tracks (track_name, track_path, artist_id, album_id) values ( ?, ?, ?, ? )"
#                 run_query(cur, con, query, (track, artist_info[artist]['album_years'][i], artist_id))
#                 print(">> Entries added:", album, " - ", artist_info[artist]['album_years'][i], " - ", artist_id)                  
#             except Exception as exc: print("! Exception {}".format(exc))
    
    
con, cur = connect_sqlite("../database/azlyrics.db")  
store_data(con, cur) 
con.close()


! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
! Exception UNIQUE constraint failed: albums.album_name, albums.artist_id
Club Foot
5
Processed Beats
5
Reason Is Treason
5
I.D.
5
Orange (Interlude)(Instrumental)
5
L.S.F. (Lost Souls Forever)
5
Running Battle
5
Test Transmission
5
Pinch Roller (Interlude)(Instrumental)
5
Cutt Off
5
Butcher Blues
5
Ovary Stripe(Instrumental)
5
U Boat
5
Reason Is Treason (Jacknife Lee Version)(Hidden Track)
5
Empire
5
Shoot The Runner
5
Last Trip (In Flight)
5
Me Plus One
5
Sun Rise Light Flies

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Generate Database Docs </span>

In [6]:
def generate_docs(db, docs, title):
    file = open(docs, "w+")
    file.write(title + "\n")
    table_tmp = "|{}|{}|\n| :-: | :-: |\n".format("Column", "Description")

    con, cur = connect_sqlite(db)
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    for table in cur.fetchall():
        table_name = table[0].upper()        
        file.write("### {}\n".format(table_name))
        file.write(table_tmp)
        
        cur.execute("SELECT * FROM {};".format(table_name))
        columns = [description[0] for description in cur.description]
        
        for column in columns:
            file.write("| {} | |\n".format(column))
    print(">> Docs generated.")
    file.close()    
    
generate_docs("../database/azlyrics.db", "../docs/AZlyrics_template.md", "# AZlyrics Database Documentation")
    

>> Docs generated.


In [4]:
### IRRELEVANT, PREVIOUS CODES  ---------------------------------

In [1]:
import string
import pendulum
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from bs4 import BeautifulSoup
from ipywidgets import Layout
from IPython.display import display, clear_output


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Load Artists Library </span>

In [2]:
%run py_utils/generic_utils.ipynb

def load_artist_from_json():
    start_urls = []
    path = '../datasets/artists_data.json'
    all_artists = load_json(path)
    return all_artists


In [5]:
all_artists = load_artist_from_json()

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Artist Selection Interface </span>

In [3]:
%run py_utils/widgets_utils.ipynb

def get_artists():    
    all_artists = load_artist_from_json()
    max_cache = 3
    alphabets = [letter for letter in string.ascii_uppercase]
    alphabets.append('#')
    user_selection = []
    
    def alphabet_based_artists(change):
        flag = 0
        artists = []
        artists_alt = []
        selected_alphabet = ''.join(change.new)         
        
        for i in all_artists['Artists']:
            if i[0].upper() == selected_alphabet:
                artists.append(i)
                flag = 1
                
            elif i[0].upper() not in alphabets or i[0] == '#':                
                artists_alt.append(i)
        
        if flag: artist_dropdown.options = artists; artist_dropdown.value = artists[0]
        else: artist_dropdown.options = artists_alt; artist_dropdown.value = artists_alt[0]
    
    def display_selected_artist(change):
        selection = ''.join(change.new)
        widget_output(output, selection)
        artist_dropdown.value = selection
            
    def retrieve_data(_): 
        if len(user_selection) < max_cache:
            comment = artist_dropdown.value+" saved."
            user_selection.append(artist_dropdown.value)
            widget_output(output, comment)
            
        else:
            comment = "Gold membership is required to select more than 3 artists."
            widget_output(output, comment)
    
    def clear_data(_):
        user_selection.clear()
        comment = "Selection Cleared."
        widget_output(output, comment)
    
    output = create_output()
    alphabet_dropdown = assign_dropdown(alphabets, "Alphabet", arg_function=alphabet_based_artists, observe=True)
    artist_dropdown = assign_dropdown(all_artists, "Artist", arg_function=display_selected_artist, observe=True)    
    dropdowns = create_gui(alphabet_dropdown, artist_dropdown, num=[0,1], wd='300', widget_type='H', gui_display=0)
    
    select_button = create_button("Select Artist", arg_function=retrieve_data)
    clear_button = create_button("Clear Selection", arg_function=clear_data)
    buttons = create_gui(select_button, clear_button, num=[0,1], wd='300', widget_type='H', gui_display=0)    
    create_gui(output, dropdowns, buttons, widget_type='V', gui_display=1) 
    
    return user_selection
    

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Generate URLs for the specified Artists from Memory </span>

In [7]:
%run py_utils/azlyrics_utils.ipynb

def search_dict(user_selection, all_artists):
    start_urls = match_artists(user_selection, all_artists)
    return start_urls

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Crawl and Scrape tracks from User Artists </span>

In [6]:
%run crawlers/tracks_crawler.ipynb
%run py_utils/sqlite_utils.ipynb

def run_tracks_crawler(start_urls, user_selection):
    db_soundtracks = 'db_soundtracks'  
    soundtrack_table = 'soundtrack_table'  
    db_soundtracks_path = f'../database/{db_soundtracks}.db' 
    
    for artist in user_selection:
        artist += ' Lyrics'
        try:
            selection_search = search_data_from_db(artist, db_soundtracks_path, soundtrack_table, 'artists')
            
        except Exception as exc:
            selection_search = 0
            print(exc) 
    
        if selection_search == 0:
            tracks_crawler(start_urls)

        else:
            print(" Skipping the Scraping Procedure.")


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Load discography from Memory </span>

In [10]:
def load_discography():
    json_path = '../datasets/discography_data.json'
    discography = load_json(json_path)
    discography.keys()
    return discography

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Track Selection Interface </span>

In [11]:
%run py_utils/generic_utils.ipynb
%run py_utils/azlyrics_utils.ipynb
%run py_utils/sqlite_utils.ipynb

def get_discography(user_selection):    
    discography = load_discography()
    max_cache = 3
    user_artist = []
    user_album = []
    user_track = []
    
    db_soundtracks = 'db_soundtracks'  
    soundtrack_table = 'soundtrack_table'  
    db_soundtracks_path = f'../database/{db_soundtracks}.db'  

    def artist_based_albums(change):
        selected_artists = ''.join(change.new)+' Lyrics'
        
        albums_list = []        
        query_albums = f"""select distinct albums from {soundtrack_table} where artists='{selected_artists}'"""
        db_data = get_data_from_db(db_soundtracks_path, query_albums)
        albums_list = [album[0] for album in db_data]
                
        if albums_list == []: 
            print(f"! {selected_artists} does not exist in the database. Accessing memory.")  
            albums_list = get_albums(discography, selected_artists) 
        
        album_dropdown.options = albums_list 
        album_dropdown.value = albums_list[0]
                
    def album_based_tracks(change):
        selected_albums = ''.join(change.new)          
        tracks_list = []
        query_tracks = f"""select distinct tracks from {soundtrack_table} where albums='{selected_albums}' 
                        and artists ='{artist_dropdown.value} Lyrics'"""
        db_data = list(get_data_from_db(db_soundtracks_path, query_tracks))
        tracks_list = [album[0] for album in db_data]
                
        if tracks_list == []:
            print(f"! {selected_albums} does not exist in the database. Accessing memory.")  
            tracks_list = get_tracks(discography, selected_albums) 

        track_dropdown.options = tracks_list
        track_dropdown.value = tracks_list[0]
    
    def display_selected_track(change):
        selection = change.new
        widget_output(output, selection)
        track_dropdown.value = selection        
        
    def retrieve_track(_):            
        if len(user_artist) < max_cache:
            comment = track_dropdown.value + " saved."
            selected_artist = ''.join(artist_dropdown.value)
            selected_album = ''.join(album_dropdown.value) if selected_artist else None
            selected_track = ''.join(track_dropdown.value) if selected_album else None
            
            user_artist.append(selected_artist)
            user_album.append(selected_album)
            user_track.append(selected_track)
            widget_output(output, comment)
        else:
            comment = "Note: Free Membership only allows up to 3 songs."
            widget_output(output, comment)            

    def reset_selection(_):
        user_artist.clear()
        user_album.clear()
        user_track.clear()
        comment = "Selection Cleared."
        widget_output(output, comment)  
        
    def refresh_selection(_):
        selected_artists = user_selection[0]+' Lyrics'        
        albums_list = []        
        query_albums = f"""select distinct albums from {soundtrack_table} where artists='{selected_artists}'"""
        db_data = get_data_from_db(db_soundtracks_path, query_albums)
        albums_list = [album[0] for album in db_data]
                
        if albums_list == []: 
            print(f"! {selected_artists} does not exist in the database. Accessing memory.") 
            albums_list = get_albums(discography, selected_artists) 

        album_dropdown.options = albums_list 
        album_dropdown.value = albums_list[0]     
    
    output = create_output()    
    artist_dropdown = assign_dropdown(user_selection, "Select Artists", arg_function=artist_based_albums, 
                                                observe=1)    
    album_dropdown = assign_dropdown([], "Select Album", arg_function=album_based_tracks, observe=1)    
    track_dropdown = assign_dropdown([], "Select Tracks", arg_function=display_selected_track, observe=1)
    button_track = create_button("Select Track", arg_function=retrieve_track)
    button_reset = create_button("Clear Selection", arg_function=reset_selection)    
    button_refresh = create_button("Refresh", arg_function=refresh_selection)
    
    dropdowns = create_gui(artist_dropdown, album_dropdown, track_dropdown, num=[0,1,2], wd='350', 
                           widget_type='H', gui_display=0)
    buttons = create_gui(button_track, button_reset, button_refresh, num=[0,1,2], wd='250', widget_type='H', gui_display=0)
    create_gui(dropdowns, buttons, output, widget_type='V', gui_display=1)
    
    return user_track, user_artist, user_album
    

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Generate URLs for the selected tracks </span>

In [1]:
%run py_utils/azlyrics_utils.ipynb

def generate_urls(user_tracks):
    print(discography)
    start_urls= []

    for i, track in enumerate(user_tracks):
        print(user_album[i])
        start_urls.append(get_url(discography, track, user_artist[i]+' Lyrics', user_album[i]))

    return start_urls

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Crawl and Sccrape lyrics from user tracks </span>

In [13]:
%run crawlers/lyrics_crawler.ipynb

def lyrics_scrape(start_urls, user_track):
    db_lyrics = 'db_lyrics'  
    lyrics_table = 'lyrics_table'  
    db_lyrics_path = f'../database/{db_lyrics}.db'  
    
    for track in user_track:
        front = '"'
        track = front+track+'" lyrics'
        
        try:
            selection_search = search_data_from_db(track, db_lyrics_path, lyrics_table, 'tracks')
            
        except Exception as exc:                        
            selection_search = 0
            print(exc) 
        
    if selection_search == 0:
        lyrics_crawler(start_urls)
    
    else:
        print("Skipping the Scraping Procedure.")
        

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Load Lyrics from Memory </span>

In [14]:
%run py_utils/generic_utils.ipynb

def load_lyrics():
    json_path = '../datasets/lyrics_data.json'
    track_lyrics = load_json(json_path)

    track_lyrics.keys()
    return track_lyrics

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Display Lyrics and Artist Information </span>

In [15]:
%run py_utils/azlyrics_utils.ipynb
%run py_utils/widgets_utils.ipynb
%run py_utils/sqlite_utils.ipynb


def get_lyrics(track_lyrics, user_track):
    lyrics_table = 'lyrics_table'  
    db_lyrics_path = f'../database/db_lyrics.db'  
    selection_size = len(track_lyrics['Tracks'])   
    
    def select_track(change):        
        selected_track = ''.join(change.new) 
        print(selected_track)
        
        track_dropdown.value = selected_track
        widget_output(output, selected_track)
    
    def display_lyrics(_):
        tracks = []
        artists = []
        lyrics = []
        albums = []
        credits = []
        
        selected_lyrics = ''.join(track_dropdown.value)  
        selected_lyrics = '"'+selected_lyrics+'" lyrics'
        print(selected_lyrics)
        
#         list_arg = [[0,1,2,3,4], selected_lyrics, db_lyrics_path, lyrics_table, 'tracks']        
#         tracks, artists, lyrics, albums, credits = get_track_lyrics_database(list_arg) 
               
        query_albums = f"""select * from {lyrics_table}"""
        print(query_albums)
        db_data = get_data_from_db(db_lyrics_path, query_albums)
        for row in db_data:
#             print(row[0], selected_lyrics)
            if row[0] == selected_lyrics:
                tracks.append(row[0]) 
                artists.append(row[2]) 
                lyrics.append(row[1]) 
                albums.append(row[3]) 
                credits.append(row[4]) 
        
        print(lyrics)
        
        if len(tracks) == 0:
            get_track_lyrics_dataset(output, track_lyrics, selected_lyrics)  
            
        else:
            result = [tracks, lyrics, artists, albums, credits]
            widget_output_iteration(output, result)

    output = create_output() 
    track_dropdown = assign_dropdown(user_track, "Select Tracks", arg_function=select_track, observe=1)   
    lyrics_button = create_button("Display Lyrics", arg_function=display_lyrics)  
    
    dropdown_button = create_gui(track_dropdown, lyrics_button, widget_type='H', num=[0,1], wd='400', gui_display=0)
    create_gui( dropdown_button, output, widget_type='V', num=[1], wd='400', gui_display=1)    


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Database Review </span>

In [16]:
%run py_utils/sqlite_utils.ipynb

def diagnose_db(database, table, column_index, db_path):
    db_path = db_path+database+'.db'
    connection = create_connection(db_path)
    cursor = create_cursor(connection)
    
    artist_list = []
    tracks_list = []
    data = cursor.execute(f'''select * from {table}''')

    for row in data:
        if row[column_index] not in artist_list:
            artist_list.append(row[column_index])
    print(artist_list)
